# 

In [1]:
# Project CE-901- MSC Dissertation 
# Author- Subhasmit Halder,Id-SH21442, Registration No-2100858
# Object detection in 3D work space to determine the the depth of the objects for robotics manipulation using 2 webcams by producing stereovision
# using 2 webcams by producing stereovision.
# Also, for user to select the object for manipulation would be taken through eyeblink detection in a multimodal approach.
# The multimodal approach consists of eyeblink detetion using image processing and BCI(Predcition of eeg signal with CNN,LSTM and Ensemble)

####  All the Required libraries to download:

In [2]:
# !pip install opencv-python
# !pip install imutils
# !pip install cmake
# !pip install mediapipe
# !pip install cvzone
# !pip install cython numpy


####  Importing all the Required Libraries:

In [3]:
import cv2
import numpy as np
import time
import numpy as np
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
import imutils




#### Initialising the YOLO version for Object detection and setting up the COCO Dataset which was used to train the YOLO:

In [4]:

classfile='coco.names' # The Dataset Annotation
with open(classfile,'rt') as f:
    classname=f.read().rstrip('\n').split('\n')
print(len(classname))
net=cv2.dnn.readNetFromDarknet('yolov3.cfg','yolov3.weights') # We are using the Darknet YOLO
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)



80


#### Marking all the irrelevant object to avoid unnecessary prediction of irrelevant object:


In [5]:
# This application is a user friendly and safe application and we are rmoving unnecassary object which are not available in
# household and also removing object which might pose safety issues to the client for eg: knife, toaster etc.
irrelevant_object=['person',
'bicycle',
'car',
'motorbike',
'aeroplane',
'bus',
'train',
'truck',
'boat',
'stopsign',
'parkingmeter',
'bench',
'bird',
'cat',
'dog',
'horse',
'sheep',
'cow',
'elephant',
'bear',
'zebra',
'giraffe',
'backpack',
'umbrella',
'tie',
'frisbee',
'skis',
'snowboard',
'kite',
'baseballbat',
'baseballglove',
'skateboard',
'surfboard',
'tennisracket',
'knife',
'chair',
'sofa',
'bed',
'diningtable',
'toilet',
'microwave',
'oven',
'toaster',
'refrigerator',
'scissors']


####  Here we are finding the object predicted by the yolo and extracting the bounding box and 2D dimension of the object with higher confidence value.

In [6]:
def findobjects(outputs,img):
    ht,wt,ct=img.shape
    bbox=[]
    classids=[]
    confs=[]
    for output in outputs:
        for det in output:
            scores=det[5:]
            classid=np.argmax(scores)
            confidence=scores[classid]
            if confidence > 0.6:
                w,h=int(det[2]*wt),int(det[3]*ht)
                x,y=int((det[0]*wt)-w/2),int((det[1]*ht)-h/2)
                bbox.append([x,y,w,h])
                classids.append(classid)
                confs.append(float(confidence))
    indices=cv2.dnn.NMSBoxes(bbox,confs,0.7,0.2)
    print(indices)
    xl,yl,wl,hl,name=[],[],[],[],[]
    for i in indices:
        box=bbox[i]
        if classname[classids[i]].lower() not in irrelevant_object:
            x,y,w,h= box[0],box[1],box[2],box[3]
            xl.append(box[0])
            yl.append(box[1])
            wl.append(box[2])
            hl.append(box[3])
            name.append(classname[classids[i]].lower())
#             cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
#             cv2.putText(img,f'{classname[classids[i]].upper()} { int(confs[i]*100)} %',(x,y-5),cv2.FONT_ITALIC,0.6,(255,0,255))
#             cv2.putText(img,f'{"x - "+str(x).upper()+"y - "+str(y).upper()} { int(confs[i]*100)}',(x,y-10),cv2.FONT_ITALIC,0.6,(255,0,255))
        
    return xl,yl,wl,hl,name

#### Here we are using facemesh detector to find the eyelid coordinate of the face of the patient and the idetifiying the blink.

In [7]:

ids=[21,22,23,25,159,130,243]

def eyeblinkdetection(img3):
    faceimg,face=facedetector.findFaceMesh(img3,draw=False)
    print("blink")
    if face:
        facepoint=face[0]
#         for id in ids:
            
        leftup=facepoint[159]
        leftdown=facepoint[23]
        leftleft=facepoint[130]
        leftright=facepoint[243]
#           cv2.circle(img,facepoint[id],2,(0,0,255))
        leftlengthver, _ =facedetector.findDistance(leftup,leftdown)
        leftlengthhor, _ =facedetector.findDistance(leftleft,leftright)
        
        print("blink",leftlengthver/leftlengthhor)
       
#         cv2.imshow('blink',img3)
#         cv2.waitKey(2000)
       
        if (leftlengthver/leftlengthhor)*100 <= 28:
        
            return True
        else:
            return False
            

#### Here we are using Triangulation and Trigonometric formulae to measure the Depth and Height

In [8]:



def find_depth(right_x, left_x, frame_right, frame_left, baseline,f, alpha):
    # CONVERT FOCAL LENGTH f FROM [mm] TO [pixel]:
    height_right, width_right, depth_right = frame_right.shape
    height_left, width_left, depth_left = frame_left.shape

    if width_right == width_left:
        f_pixel = (width_right * 0.5) / np.tan(alpha * 0.5 * np.pi/180)

    else:
        print('Left and right camera frames do not have the same pixel width')

    x_right = right_x
    x_left = left_x

    # CALCULATE THE DISPARITY:
    disparity = x_left-x_right      #Displacement between left and right frames [pixels]
    

    print("disparity ",disparity)
    # CALCULATE DEPTH z:
    zDepth = (baseline*f_pixel)/disparity             #Depth in [cm]
    #Calculate height
    yHeight=zDepth*np.tan(alpha* 0.5 * np.pi/180)     #Height in [cm]

    return abs(zDepth),abs(yHeight)




#### This is the Handle program that is calling all the functionalities

In [9]:




frame_rate = 30    #Camera frame rate (maximum at 30 fps)
B = 12             #Distance between the cameras [cm]
f = 2               #Camera lense's focal length [mm]
alpha = 40        #Camera field of view in the horisontal plane [degrees]

count=0

#Variable store execution state
first_read = True

#Starting the video capture for all the purposes

cap_right = cv2.VideoCapture(1, cv2.CAP_DSHOW)                    
cap_left =  cv2.VideoCapture(2, cv2.CAP_DSHOW)
cap_blink = cv2.VideoCapture(0,cv2.CAP_DSHOW)
    



facedetector=FaceMeshDetector(maxFaces=1) #since we have only the patient face we are putting maxfaces=1.

ids=[21,22,23,25,159,130,243]  # The point arround the right eye of the patient.

# The Main body where all the components are called.

while True:
    success, img=cap_right.read()
    success, img1=cap_left.read()
    
    
    blob=cv2.dnn.blobFromImage(img,1/255,(320,320),[0,0,0],1,crop=False)
    blob1=cv2.dnn.blobFromImage(img1,1/255,(320,320),[0,0,0],1,crop=False)
    
    net.setInput(blob1)
    
    layerNames1=net.getLayerNames()
    val=net.getUnconnectedOutLayers()
    
    outputNames1=[]
    net.setInput(blob1)
    for i in val:
        
        outputNames1.append(layerNames1[i-1])
   
    outputs=net.forward(outputNames1)
    
    
    x1,y1,w1,h1,name1=findobjects(outputs,img1)
    layerNames=net.getLayerNames()
    net.setInput(blob)
    outputNames=[]
    for i in net.getUnconnectedOutLayers():
        
        outputNames.append(layerNames[i-1])
   
    outputs=net.forward(outputNames)
    
    x,y,w,h,name=findobjects(outputs,img)
   
    print("The Object Identified By Camera1- ",name)
    print("The Object Identified By Camera2-",name1)
    

    for i in range(len(name)):
        if name[i] in name1:
            i2=name1.index(name[i])
            imgnew=img.copy()
            imgnew1=img1.copy()
            print(len(x),len(y),len(w),len(h),len(name))
            cv2.rectangle(imgnew,(x[i],y[i]),(x[i]+w[i],y[i]+h[i]),(255,255,0),5)
            cv2.imshow('Captureimage',imgnew)
            cv2.rectangle(imgnew1,(x1[i2],y1[i2]),(x1[i2]+w1[i2],y1[i2]+h1[i2]),(255,255,0),5)
            cv2.imshow('Captureimage1',imgnew1)
            
            

            d,ht=find_depth(x[i],x1[i2],imgnew,imgnew1,B,f,alpha)
            
            
            print(name[i],name1[i2],"The depth of the object-",d,"Height-",ht)
            
            success, img3=cap_blink.read()
            decision=eyeblinkdetection(img3)
           
            if decision:
                print("object selected is ",name[i])
            else:
                print("no object selected ")
            cv2.waitKey(3000)
            
        
        
        
        
    
    cv2.imshow('Captureimage',img)
    cv2.imshow('Captureimage1',img1)
    
   
    a = cv2.waitKey(3000)

    if(a==ord('q')):
        break
    elif(a==ord('s') and first_read):
        #This will start the detection
        first_read = False
cap_left.release()
cap_right.release()
cv2.destroyAllWindows()


()
()
The Object Identified By Camera1-  []
The Object Identified By Camera2- []
()
()
The Object Identified By Camera1-  []
The Object Identified By Camera2- []
[0]
[0 2]
The Object Identified By Camera1-  ['cup']
The Object Identified By Camera2- ['cup']
1 1 1 1 1
disparity  -363
cup cup The depth of the object- 29.064223941338152 Height- 10.578512396694213
blink
blink 0.350347944635714
no object selected 
[0 3 4]
[1 3]
The Object Identified By Camera1-  ['cup']
The Object Identified By Camera2- ['cup']
1 1 1 1 1
disparity  -302
cup cup The depth of the object- 34.93481222087997 Height- 12.715231788079471
blink
no object selected 
[0 4 5]
[0]
The Object Identified By Camera1-  []
The Object Identified By Camera2- ['cup']
[0]
[0]
The Object Identified By Camera1-  []
The Object Identified By Camera2- ['cup']
[0]
[0 1]
The Object Identified By Camera1-  ['cup']
The Object Identified By Camera2- ['cup']
1 1 1 1 1
disparity  -276
cup cup The depth of the object- 38.22577279241214 Height-

disparity  -251
bottle bottle The depth of the object- 42.03312068010259 Height- 15.298804780876495
blink
blink 0.304401955412109
no object selected 
[5 2 3]
[0 3]
The Object Identified By Camera1-  ['bottle', 'cup']
The Object Identified By Camera2- ['bottle', 'cup']
2 2 2 2 2
disparity  -229
bottle bottle The depth of the object- 46.07123707731768 Height- 16.768558951965066
blink
no object selected 
2 2 2 2 2
disparity  -292
cup cup The depth of the object- 36.13120989967722 Height- 13.150684931506849
blink
no object selected 
[5 1 2]
[2 3]
The Object Identified By Camera1-  ['bottle']
The Object Identified By Camera2- ['bottle', 'cup']
1 1 1 1 1
disparity  -201
bottle bottle The depth of the object- 52.48912084928234 Height- 19.104477611940297
blink
blink 0.21966088569534453
object selected is  bottle
[3 1]
[0]
The Object Identified By Camera1-  ['bottle']
The Object Identified By Camera2- ['bottle', 'cup']
1 1 1 1 1
disparity  -200
bottle bottle The depth of the object- 52.75156645

[0 2]
[7 3]
The Object Identified By Camera1-  ['bottle', 'cup']
The Object Identified By Camera2- ['bottle', 'cup']
2 2 2 2 2
disparity  -216
bottle bottle The depth of the object- 48.84404301252662 Height- 17.77777777777778
blink
blink 0.18318582636182792
object selected is  bottle
2 2 2 2 2
disparity  -267
cup cup The depth of the object- 39.51428198766198 Height- 14.38202247191011
blink
no object selected 
[0 2]
[7 3]
The Object Identified By Camera1-  ['bottle', 'cup']
The Object Identified By Camera2- ['bottle', 'cup']
2 2 2 2 2
disparity  -216
bottle bottle The depth of the object- 48.84404301252662 Height- 17.77777777777778
blink
blink 0.2617119612951069
object selected is  bottle
2 2 2 2 2
disparity  -268
cup cup The depth of the object- 39.366840636961754 Height- 14.328358208955224
blink
no object selected 
[0 2]
[7 3]
The Object Identified By Camera1-  ['bottle', 'cup']
The Object Identified By Camera2- ['bottle', 'cup']
2 2 2 2 2
disparity  -215
bottle bottle The depth of t

In [13]:
# This is for creating the requirement.txt file
# !pip install pipreqsnb
# !pipreqsnb .


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pipreqsnb: filename=pipreqsnb-0.2.4-py3-none-any.whl size=4126 sha256=cd17cfde1b8918274c47e6e68173a0ad7816352b2a49a62ff9cb884390ea43e3
  Stored in directory: c:\users\subha\appdata\local\pip\cache\wheels\c3\1b\47\ca78a979321cb4bf9e812da5157fb754712f43d094bc60bf61
Successfully built pipreqsnb
pipreqs  .

INFO: Successfully saved requirements file in .\requirements.txt
